#AtomMan LAMMPS atom_dump Demonstration

The atomman package was designed to allow for the creation and analysis of large scale atomistic simulations.  To help facilitate this, the atomman.lammps module contains a number of useful functions and classes that allow for atomman to interact with the LAMMPS molecular dynamics software.

Interactions with LAMMPS atomic dump files is handled with the atomman.lammps.atom_dump module.  This module contains two functions, load and dump, that allow for dump files to be read into atomman and written to a file, respectively.  With both functions, the same json/xml data model is used to define how the LAMMPS atomic attributes correspond to the atomman atomic properties. 

Library imports

In [1]:
import atomman as am
import atomman.lammps as lmp
import numpy as np
import os

In [2]:
#create small demonstration system (GaAs unit cell)
prop_dict = {'atype':  [1, 1, 1, 1, 2, 2, 2, 2],
             'pos':   [[0.00, 0.00, 0.00],
                       [0.50, 0.00, 0.50],
                       [0.50, 0.50, 0.00],
                       [0.00, 0.50, 0.50],
                       [0.25, 0.25, 0.25],
                       [0.75, 0.75, 0.25],
                       [0.25, 0.75, 0.75],
                       [0.75, 0.25, 0.75]],
            'scalar':  [0.,1.,2.,3.,4.,5.,6.,7.],
            'vector': [[1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4],
                       [1, 2, 3, 4]],
            'tensor':[[[12.4, 10.1], [10.1, 3.0]],
                      [[22.0,  9.8], [ 9.8, 1.0]],
                      [[11.0, 13.1], [13.1, 3.4]],
                      [[41.0, 18.9], [18.9, 2.1]],
                      [[16.7, 14.7], [14.7, 1.6]],
                      [[19.2,  8.9], [ 8.9, 2.1]],
                      [[12.0, 13.1], [13.1, 2.4]],
                      [[13.2, 11.2], [11.2, 1.5]]]}


atoms = am.Atoms(natoms=8, prop=prop_dict)
box = am.Box(a=5.65, b=5.65, c=5.65)
system = am.System(box=box, atoms=atoms, scale=True)
print "system.box ->"
print system.box
print
print "system.atoms ->"
print system.atoms
print 
print "system.atoms_prop(key='scalar') ->"
print system.atoms_prop(key='scalar')
print
print "system.atoms_prop(key='vector') ->"
print system.atoms_prop(key='vector')
print
print "system.atoms_prop(key='tensor') ->"
print system.atoms_prop(key='tensor')
print

system.box ->
avect =  [ 5.650,  0.000,  0.000]
bvect =  [ 0.000,  5.650,  0.000]
cvect =  [ 0.000,  0.000,  5.650]
origin = [ 0.000,  0.000,  0.000]

system.atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.825 |   0.000 |   2.825
      2 |       1 |   2.825 |   2.825 |   0.000
      3 |       1 |   0.000 |   2.825 |   2.825
      4 |       2 |   1.413 |   1.413 |   1.413
      5 |       2 |   4.238 |   4.238 |   1.413
      6 |       2 |   1.413 |   4.238 |   4.238
      7 |       2 |   4.238 |   1.413 |   4.238

system.atoms_prop(key='scalar') ->
[ 0.  1.  2.  3.  4.  5.  6.  7.]

system.atoms_prop(key='vector') ->
[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]

system.atoms_prop(key='tensor') ->
[[[ 12.4  10.1]
  [ 10.1   3. ]]

 [[ 22.    9.8]
  [  9.8   1. ]]

 [[ 11.   13.1]
  [ 13.1   3.4]]

 [[ 41.   18.9]
  [ 18.9   2.1]]

 [[ 16.7  14.7]
  [ 14.7   1.6]]



##1. atom_dump.dump()

The atom_dump.dump() function writes a system's information to a LAMMPS dump style file. The function arguments are:

- __fname__ = name (and location) of the file to write to. Required.

- __system__ = the atomman.System to extract values from. Required.

- __prop_info__ = the data model defining how to relate the LAMMPS atomic attributes to the atomman atomic properties. If not specified, will try reading [fname].json.  If that doesn't work, a defaut model will be created based on the system and saved to [fname].json.

- __xf__ = c-style format string to use for floating point numbers. Default is '%.13e'. 

In [3]:
#save system info to dump file
lmp.atom_dump.dump('test.dump', system, xf='%.3f')

#show contents of test.dump
with open('test.dump') as f:
    print f.read()

ITEM: TIMESTEP
0
ITEM: NUMBER OF ATOMS
8
ITEM: BOX BOUNDS pp pp pp
0.000000 5.650000
0.000000 5.650000
0.000000 5.650000
ITEM: ATOMS id type x y z scalar vector[1] vector[2] vector[3] vector[4] tensor[1][1] tensor[1][2] tensor[2][1] tensor[2][2]
1 1 0.000 0.000 0.000 0.000 1 2 3 4 12.400 10.100 10.100 3.000
2 1 2.825 0.000 2.825 1.000 1 2 3 4 22.000 9.800 9.800 1.000
3 1 2.825 2.825 0.000 2.000 1 2 3 4 11.000 13.100 13.100 3.400
4 1 0.000 2.825 2.825 3.000 1 2 3 4 41.000 18.900 18.900 2.100
5 2 1.413 1.413 1.413 4.000 1 2 3 4 16.700 14.700 14.700 1.600
6 2 4.238 4.238 1.413 5.000 1 2 3 4 19.200 8.900 8.900 2.100
7 2 1.413 4.238 4.238 6.000 1 2 3 4 12.000 13.100 13.100 2.400
8 2 4.238 1.413 4.238 7.000 1 2 3 4 13.200 11.200 11.200 1.500



__Note:__ integer values are printed as integers and all floats are printed using the specified xf format.

##2. atom_dump.load()

The atom_dump.load() function reads in a LAMMPS dump style file. The function arguments are:

- __fname__ = name (and location) of the file to write to. Required.

- __prop_info__ = the data model defining how to relate the LAMMPS atomic attributes to the atomman atomic properties. If not specified, will try reading [fname].json.  If that doesn't work, a defaut model will be created based on the system and saved to [fname].json.

The function will return all atomic data contained in an atomman.System.

__Note:__ for this demo the default test.dump.json model that was created when calling atom_dump.dump() above is automatically loaded as well. More information on the data models is in Section 3.

In [4]:
system2 = lmp.atom_dump.load('test.dump')

print "system2.box ->"
print system2.box
print
print "system2.atoms ->"
print system2.atoms
print 
print "system.atoms_prop(key='scalar') ->"
print system.atoms_prop(key='scalar')
print
print "system.atoms_prop(key='vector') ->"
print system.atoms_prop(key='vector')
print
print "system.atoms_prop(key='tensor') ->"
print system.atoms_prop(key='tensor')
print

system2.box ->
avect =  [ 5.650,  0.000,  0.000]
bvect =  [ 0.000,  5.650,  0.000]
cvect =  [ 0.000,  0.000,  5.650]
origin = [ 0.000,  0.000,  0.000]

system2.atoms ->
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.825 |   0.000 |   2.825
      2 |       1 |   2.825 |   2.825 |   0.000
      3 |       1 |   0.000 |   2.825 |   2.825
      4 |       2 |   1.413 |   1.413 |   1.413
      5 |       2 |   4.238 |   4.238 |   1.413
      6 |       2 |   1.413 |   4.238 |   4.238
      7 |       2 |   4.238 |   1.413 |   4.238

system.atoms_prop(key='scalar') ->
[ 0.  1.  2.  3.  4.  5.  6.  7.]

system.atoms_prop(key='vector') ->
[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]

system.atoms_prop(key='tensor') ->
[[[ 12.4  10.1]
  [ 10.1   3. ]]

 [[ 22.    9.8]
  [  9.8   1. ]]

 [[ 11.   13.1]
  [ 13.1   3.4]]

 [[ 41.   18.9]
  [ 18.9   2.1]]

 [[ 16.7  14.7]
  [ 14.7   1.6]]

##3. LAMMPS-dump-atoms_prop-relate Data Model

A json/xml data model is used by the atom_dump functions to define the proper transformations of the atomic properties between LAMMPS dump files and atomman Systems.  Within the data model, LAMMPS values can be defined as vector and tensor components, property names can be custom-changed, values can be specified as integers or booleans, and units can be converted.

Within the data model are three main branches: _box_prop_, _atoms_prop_ and _LAMMPS-attribute_. The _box_prop_ and _atoms_prop_ branches define the metadata associated with the Box and Atoms within the atomman.System. The _LAMMPS-attribute_ branch defines how the atomic attributes listed in the dump file relate to the atoms properties.

###3.1 The _box_prop_ branch

The _box_prop_ branch of the data model allows for the units of the System's Box to be defined.

- __['unit']__ defines the units of the box as represented in the LAMMPS dump file. If not specified, then the unit is taken to be null (i.e. no conversion is performed.)

###3.2 The atoms_prop branch

The atoms_prop branch of the data model allows for the names, shapes and data types of the various atomic properties as stored in the atomman System.

- The __prop_name__ keys specify the names of the atomic properties within the System. During load, all properties listed here will be created with initial values of zero before value assignment.

- __\[prop-name\]\['dtype'\]__ is a string specifying what data type the values of that property shoud be interpreted as, e.g. "int", "float", "bool".  If this term is not listed or has a null value then the data type is inferred from the values.

- __\[prop-name\]\['shape'\]__ can be either null, an integer, or a list of integers.  A null value creates a scalar property. An integer creates a vector property of the specified length. A list of integers creates a tensor with the specified dimensions and shape.  If this term is not listed, then it is taken to be null (property is made scalar).

###3.3 The LAMMPS-attribute branch

The LAMMPS-attribute branch of the data model defines how the atomic properties in the LAMMPS dump file should be interpreted. 

- The __attribute__ keys are the names of the atomic properties as they appear in the LAMMPS dump file.  Only __attributes__ listed will be read from/written to the dump file.

- __\[attribute\]\['relation'\]__ is a dictionary (or list of dictionaries) indicating to atomman how to interpret the associated __attribute__.   

- __\[attribute\]\['relation'\]\['prop'\]__ is the name of the __prop_name__ (as defined in the _atoms_prop_ branch) to associate with the __attribute__. This has to be specified for each relation.

- __\[attribute\]\['relation'\]\['index'\]__ is the index of the __prop_name__ to associate with the __attribute__. The value can either be null, an integer, or a list of integers depending on whether __prop_name__ is a scalar, vector or tensor.  This term can be excluded for scalar __prop_names__.

- __\[attribute\]\['relation'\]\['unit'\]__ is a string which defines the units of the __attribute__ as represented in the LAMMPS dump file. If the value of __\['unit'\]__ is "scaled", then the property will be scaled relative to the box vectors. If not specified, then the unit is taken to be null (i.e. no conversion is performed.)

###3.4 Default data model creation

The easiest way to create an instance of the data model is to first call atom_dump.load() or atom_dump.dump() without specifying a data model.  This will create and save a default data model to [fname].json.  The rules for the default conversions are as follows:

1. All unit conversions are neglected.

2. For both load() and dump(), the atoms_prop 'atype' is defined as a scalar integer, and the LAMMPS-attribute 'type' is related to 'atype'.

3. For both load() and dump(), the atoms_prop 'pos' is defined as a vector of three floats.  

4. For dump(), the LAMMPS-attributes 'x', 'y', and 'z' are related to atoms_prop 'pos'.

5. For load(), the LAMMPS-attributes related to atoms_prop 'pos' depend on what attributes are in the dump file.  The logic searches for a full set of ['x','y','z'], ['xs','ys','zs'], ['xu','yu','zu'], or ['xsu','ysu','zsu'] in this listed order.  When a full set is found, it is related to 'pos' and any additional coordinates are interpreted as additional properties. unit "scaled" is set for the -s and -su sets.

6. The data types for all additional properties are not specified (i.e. the code infers from the data).

7. For load(), all additional properties are interpreted as scalars with the exact same name.

8. For dump(), scalar properties are saved with the exact same name.  The components of vectors and tensors are all included and given names with bracketed indexes appended on. The index values in the attribute name start with 1 as opposed to 0 to be consistent with LAMMPS defined attributes.

__Note:__ The default data models created by load() and dump() are different.  However, both default models allow for System-dump conversions of all the data forwards and backwards. 

In [5]:
#show contents of the default test.dump.json data file created with dump()
with open('test.dump.json') as f:
    print f.read()

{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": null
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "scalar": {}, 
            "vector": {
                "shape": 4
            }, 
            "tensor": {
                "shape": [
                    2, 
                    2
                ]
            }
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos", 
                    "index": 0
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "index": 1
                }
            }, 
            "z": {
 

In [6]:
#Copy test.dump to test2.dump
with open('test.dump') as f1:
    with open('test2.dump', 'w') as f2:
        f2.write(f1.read())
        
#show contents of the default test2.dump.json data file created with load()
system2 = lmp.atom_dump.load('test2.dump')
with open('test2.dump.json') as f:
    print f.read()

{
    "LAMMPS-dump-atoms_prop-relate": {
        "box_prop": {
            "unit": null
        }, 
        "atoms_prop": {
            "atype": {
                "dtype": "int"
            }, 
            "pos": {
                "dtype": "float", 
                "shape": 3
            }, 
            "scalar": {}, 
            "vector[1]": {}, 
            "vector[2]": {}, 
            "vector[3]": {}, 
            "vector[4]": {}, 
            "tensor[1][1]": {}, 
            "tensor[1][2]": {}, 
            "tensor[2][1]": {}, 
            "tensor[2][2]": {}
        }, 
        "LAMMPS-attribute": {
            "type": {
                "relation": {
                    "prop": "atype"
                }
            }, 
            "x": {
                "relation": {
                    "prop": "pos", 
                    "index": 0
                }
            }, 
            "y": {
                "relation": {
                    "prop": "pos", 
                    "index": 1


###3.5 Modifying the data model

After creating a default data model for your data, you can open the .json file in a text editor and add any specifications you want.  A number of examples are listed below.

####3.5.1 Changing property names

This data model allows for property names to be freely changed when loading or dumping a dump file.

_Example: Renaming a LAMMPS variable name._ Say you defined a per-atom hydrostatic pressure as c_press in LAMMPS and want it to simply be called pressure.  Just define pressure in atoms_prop: 

and define the appropriate LAMMPS-attribute relation:

####3.5.2 Defining appropriate units

The units of a LAMMPS-attribute can be defined in the data model such that:

- appropriate unit conversions can be done in atomman on loaded properties calculated by LAMMPS, and 

- properties calculated in atomman can be exported to a dump file in any compatible units.

_Example: Reading value in with appropriate units._ Say LAMMPS calculated a pressure in bars.  In the data model, you can modify the LAMMPS-attribute term associated with the pressure like:

_Example: exporting value with specific units_. Once read in, the pressure can then be exported (to the same or a different dump file) in GPa by changing the model again. A simple and dirty way is to simply change the unit value before calling dump(): 

A better way is to retain the value in the original units, and simply add a new relation with a different (more descriptive) attribute name:

####3.5.3 Define vector and tensor relationships

The relations also allow for vectors and tensors in atomman to be composed from components of dump file attributes.  The simplest examples of these to see is with the default dump() data model shown in section 3.4.

_Example: More complex tensor definitions._ Perhaps you have a per-atom stress value and only apply a force along the [100] x-direction of a cubic system.  In this case, only the xx and xy components are unique.  You can define the relations such that the full 3x3 tensor exists in atomman, but only the two unique components are listed in the dump file: 

The LAMMPS compute stress/atom calculates the virial stress where [1] is the xx component and [4] is the xy component.  We can then build the stress tensor such that stress[0,0] = c_stress_atom[1], and stress[0,1] = stress[1,0] = stress[0,2] = stress[2,0] = c_stress_atom[4].  All non-defined components are automatically set to zero.  

In [7]:
#file cleanup
os.remove('test.dump')
os.remove('test2.dump')
os.remove('test.dump.json')
os.remove('test2.dump.json')